In [1]:
%%capture
!pip install titanoboa
!pip install curve_dao

In [2]:
import boa
import curve_dao
import math
import datetime

ModuleNotFoundError: No module named 'google.colab'

In [3]:
TIME_FORMAT = "%Y-%m-%d %H:%M:%S%z"

def get_datestring(ts):
    dt = datetime.datetime.fromtimestamp(ts, tz=datetime.timezone.utc)
    return dt.strftime(TIME_FORMAT)

ALCHEMY = 'ALCHEMY_KEY'
PINATA_KEY = 'PINATA_KEY'

boa.env.fork(ALCHEMY)

NameError: name 'boa' is not defined

In [4]:
# Set up constants
VOTE_CREATOR_SIM = "0x989AEb4d175e16225E39E87d0D97A3360524AD80"
POOL_ADDRESS = "0xc907ba505c2e1cbc4658c395d4a2c7e6d2c32656"
RAMP_TIME_WEEKS = 1

# new parameters
NEW_A = 3000000
NEW_GAMMA = 63000000000000000
NEW_MID_FEE = 3000000
NEW_OUT_FEE = 15000000
NEW_FEE_GAMMA = 30000000000000000
NEW_ALLOWED_EXTRA_PROFIT = 10000000000
NEW_ADJUSTMENT_STEP = 5500000000000
NEW_MA_TIME = 601

# proposal time
PROPOSAL_TIME_WEEKS = 1

In [5]:
# Load the TwocrptoNG contract
pool = boa.from_etherscan(
    POOL_ADDRESS, name="TwocrptoNG", api_key="ETHERSCAN_API_KEY"
)

NameError: name 'boa' is not defined

In [6]:
def update_twocrypto_ng(
        pool,
        ramp_time_weeks,
        new_A,
        new_gamma,
        new_mid_fee,
        new_out_fee,
        new_fee_gamma,
        new_allowed_extra_profit,
        new_adjustment_step,
        new_ma_time,
        proposal_time_weeks,
):
    SECONDS_PER_WEEK = 7 * 24 * 60 * 60

    ramp_time_seconds = int(ramp_time_weeks * SECONDS_PER_WEEK)
    proposal_time_seconds = int(proposal_time_weeks * SECONDS_PER_WEEK)

    current_time = boa.env.evm.patch.timestamp
    future_A_time = current_time + ramp_time_seconds + proposal_time_seconds

    pool_address = pool.address.strip()
    actions = [
        (pool_address, "ramp_A_gamma", new_A, new_gamma, future_A_time),    # adjust A and gamma
        (pool_address, "apply_new_parameters", new_mid_fee, new_out_fee, new_fee_gamma, new_allowed_extra_profit, new_adjustment_step, int(new_ma_time / math.log(2)))
    ]

    current_A = pool.A()
    current_gamma = pool.gamma()
    current_mid_fee = pool.mid_fee()
    current_out_fee = pool.out_fee()
    current_fee_gamma = pool.fee_gamma()
    current_allowed_extra_profit = pool.allowed_extra_profit()
    current_adjustment_step = pool.adjustment_step()
    current_ma_time = pool.ma_time()

    ramp_start_time = future_A_time - ramp_time_seconds
    ramp_start_datestring = get_datestring(ramp_start_time)
    ramp_end_datestring = get_datestring(future_A_time)

    description = (
        f"Update TwocryptoNG parameters for pool {pool_address} with: "
        f"mid_fee from {current_mid_fee} -> {new_mid_fee}, "
        f"out_fee from {current_out_fee} -> {new_out_fee}, "
        f"fee_gamma from {current_fee_gamma} -> {new_fee_gamma}, "
        f"allowed_extra_profit from {current_allowed_extra_profit} -> {new_allowed_extra_profit}, "
        f"adjustment_step from {current_adjustment_step} -> {new_adjustment_step}, "
        f"ma_time from {current_ma_time} -> {new_ma_time}, "
        f"amplification factor from {current_A} -> {new_A} ramped over {ramp_time_weeks} weeks "
        f"gamma from {current_gamma} -> {new_gamma} ramped over {ramp_time_weeks} weeks "
        f"starting on {ramp_start_datestring} and ending on {ramp_end_datestring}."
    )

    return actions, description

In [7]:
# Generate proposal actions and description
actions, description = update_twocrypto_ng(
    pool,
    RAMP_TIME_WEEKS,
    NEW_A,
    NEW_GAMMA,
    NEW_MID_FEE,
    NEW_OUT_FEE,
    NEW_FEE_GAMMA,
    NEW_ALLOWED_EXTRA_PROFIT,
    NEW_ADJUSTMENT_STEP,
    NEW_MA_TIME,
    PROPOSAL_TIME_WEEKS,
)

NameError: name 'pool' is not defined

In [8]:
SIMULATION = True

# Create and submit the proposal
if SIMULATION:
  with boa.env.prank(VOTE_CREATOR_SIM):
      vote_id = curve_dao.create_vote(
          curve_dao.get_address("ownership"),
          actions,
          description,
          "ETHERSCAN_API_KEY",
          "PINATA_KEY",
      )
  print(f"Vote ID: {vote_id}")

if not SIMULATION:
  boa.set_browser_env()
  vote_id = curve_dao.create_vote(
      curve_dao.get_address("ownership"),
      actions,
      description,
      "ETHERSCAN_API_KEY",
      "PINATA_KEY",
  )
  print(f"Vote ID: {vote_id}")


NameError: name 'boa' is not defined

In [9]:
# Simulate the proposal execution
curve_dao.simulate(vote_id, curve_dao.get_address("ownership"), "ETHERSCAN_API_KEY")

# Time travel to after the ramp period
boa.env.time_travel(seconds=60 * 60 * 24 * 8)  # 8 days (7 days ramp + 1 day buffer)

NameError: name 'curve_dao' is not defined

In [ ]:
assert pool.A() == NEW_A
assert pool.gamma() == NEW_GAMMA
assert pool.mid_fee() == NEW_MID_FEE
assert pool.out_fee() == NEW_OUT_FEE
assert pool.fee_gamma() == NEW_FEE_GAMMA
assert pool.allowed_extra_profit() == NEW_ALLOWED_EXTRA_PROFIT
assert pool.adjustment_step() == NEW_ADJUSTMENT_STEP
print(f"Expected MA time (contract value): {NEW_MA_TIME}")
print(f"Actual MA time (contract value): {pool.ma_time()}")

# Define an acceptable error margin (e.g., 0.5%)
ERROR_MARGIN = 5

# Check if the actual value is within the acceptable range
lower_bound = NEW_MA_TIME * (1 - ERROR_MARGIN)
upper_bound = NEW_MA_TIME * (1 + ERROR_MARGIN)

assert lower_bound <= pool.ma_time() <= upper_bound, (
    f"MA time {pool.ma_time()} is outside the acceptable range "
    f"[{lower_bound}, {upper_bound}]"
)

print("All parameters updated successfully!")